In [296]:
import numpy as np
import cv2
import time
import pyautogui
from PIL import ImageGrab
from numpy import ones, vstack
from numpy.linalg import lstsq
from driveKeys import PressKey, ReleaseKey, W, A, S, D
from statistics import mean

import warnings
# Ignore all warnings and never print.
warnings.filterwarnings('ignore')

In [297]:
def region_of_interest(frame, polygon):
    
    mask = np.zeros_like(frame)   
    cv2.fillPoly(mask, polygon, 255)
    
    # Returning the frame where mask pixels are non-zero
    masked_frame = cv2.bitwise_and(frame, mask)
    return masked_frame

In [298]:
def draw_lanes(frame, lines):

    try:

        ys = []  
        for i in lines:
            for ii in i:
                ys += [ii[1], ii[3]]
        min_y = min(ys)
        max_y = 600
        new_lines = []
        line_dict = {}

        for idx, i in enumerate(lines):
            for xyxy in i:
                x_coords = (xyxy[0], xyxy[2])
                y_coords = (xyxy[1], xyxy[3])
                A = vstack([x_coords, ones(len(x_coords))]).T
                m, b = lstsq(A, y_coords)[0]

                x1 = (min_y - b) / m
                x2 = (max_y - b) / m

                line_dict[idx] = [m,b,[int(x1), min_y, int(x2), max_y]]
                new_lines.append([int(x1), min_y, int(x2), max_y])

        final_lanes = {}

        for idx in line_dict:
            final_lanes_copy = final_lanes.copy()
            m = line_dict[idx][0]
            b = line_dict[idx][1]
            line = line_dict[idx][2]
            
            if len(final_lanes) == 0:
                final_lanes[m] = [ [m, b, line] ]
                
            else:
                found_copy = False

                for other_ms in final_lanes_copy:

                    if not found_copy:
                        if abs(other_ms*1.2) > abs(m) > abs(other_ms*0.8):
                            if abs(final_lanes_copy[other_ms][0][1]*1.2) > abs(b) > abs(final_lanes_copy[other_ms][0][1]*0.8):
                                final_lanes[other_ms].append([m,b,line])
                                found_copy = True
                                break
                        else:
                            final_lanes[m] = [ [m, b, line] ]

        line_counter = {}

        for lanes in final_lanes:
            line_counter[lanes] = len(final_lanes[lanes])

        top_lanes = sorted(line_counter.items(), key = lambda item: item[1])[::-1][:2]

        lane1_id = top_lanes[0][0]
        lane2_id = top_lanes[1][0]

        def average_lane(lane_data):
            x1s = []
            y1s = []
            x2s = []
            y2s = []
            
            for data in lane_data:
                x1s.append(data[2][0])
                y1s.append(data[2][1])
                x2s.append(data[2][2])
                y2s.append(data[2][3])
                
            return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

        l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
        l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

        return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2], lane1_id, lane2_id
    
    except Exception as e:
        pass

In [299]:
def bgr_to_hls(frame):
    
    hls_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
      
    # defining range for Yellow and White Color
    lower = np.array([0, 153, 153])
    upper = np.array([255, 255, 255])
    
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hls_frame, lower, upper)
    
    # Bitwise-AND mask with original frame
    hls_frame_white_yellow = cv2.bitwise_and(frame, frame, mask = mask)
    
    return hls_frame_white_yellow

In [300]:
def contrast_brightness(frame):
    
    contrast_brightness_frame = np.zeros(frame.shape, frame.dtype)
    alpha = 0   # Simple contrast control
    beta = 1    # Simple brightness control
       
    contrast_brightness_frame = cv2.convertScaleAbs(frame, alpha, beta)
    
    return contrast_brightness_frame

In [301]:
def image_processing(frame):
    
    original_frame = frame
    contrast_brightness_frame = contrast_brightness(frame)
    
    hls_frame = bgr_to_hls(contrast_brightness_frame)
    hls_to_bgr_frame = cv2.cvtColor(hls_frame, cv2.COLOR_HLS2BGR)
    
    gray_frame = cv2.cvtColor(hls_to_bgr_frame, cv2.COLOR_BGR2GRAY)
    canny_frame = cv2.Canny(gray_frame, 200, 300)    
    processed_frame = cv2.GaussianBlur(canny_frame, (5, 5), 0)
    
    polygon = np.array([[100, 450], [300, 200], [500, 200], [700, 450], [500, 450], [400, 300], [300, 450]], np.int32)
    processed_frame = region_of_interest(processed_frame, [polygon])
          
    lines = cv2.HoughLinesP(processed_frame, 1, np.pi/180, 180, np.array([]), 50, 10)
    
    m1, m2 = 0, 0
    try:        
        l1, l2, m1, m2 = draw_lanes(original_frame, lines)
        cv2.line(original_frame, (l1[0], l1[1]), (l1[2], l1[3]), [0, 255, 0], 20)
        cv2.line(original_frame, (l2[0], l2[1]), (l2[2], l2[3]), [0, 255, 0], 20)
    
    except Exception as e:
        pass
    
    try:        
        for coords in lines:
            coords = coords[0]
            try:
                cv2.line(processed_frame, (coords[0], coords[1]), (coords[2], coords[3]), [255, 0, 0], 3)
              
            except Exception as e:
                pass
                
    except Exception as e:
        pass

    return hls_to_bgr_frame, processed_frame, original_frame, m1, m2

In [302]:
# Movement
def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(S)
    ReleaseKey(D)
    ReleaseKey(W)

def left():
    PressKey(W)
    ReleaseKey(W)
    PressKey(A)
    PressKey(S)
    ReleaseKey(S) 
    ReleaseKey(D)
    #ReleaseKey(A)

def right():
    PressKey(W)
    ReleaseKey(W)
    PressKey(D)
    PressKey(S)
    ReleaseKey(S) 
    ReleaseKey(A)    
    #ReleaseKey(D)

def slow():
    ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    ReleaseKey(S)

In [303]:
def main():
    
    for i in list(range(5))[::-1]:
        print(i+1)
        time.sleep(1)
    
    last_time = time.time()
    while True:
        frame =  np.array(ImageGrab.grab(bbox = (0,40,800,630)))
        #print('Frame took {} seconds'.format(time.time()-last_time))
        last_time = time.time()
        hls_to_bgr_frame, processed_frame, original_frame, m1, m2 = image_processing(frame)
        
        hls_to_bgr_frame = cv2.resize(hls_to_bgr_frame, (550, 400))
        processed_frame = cv2.resize(processed_frame, (550, 400))
        original_frame = cv2.resize(frame, (550, 400))
                
        cv2.imshow('HLS Frame', hls_to_bgr_frame)
        cv2.imshow('Lane Lines', processed_frame)
        cv2.imshow('Weighted Frame', cv2.cvtColor(original_frame, cv2.COLOR_BGR2RGB))
              
        if m1 < 0 and m2 < 0:
            right()
            print('Right    : [ {:.4f}, {:.4f} ]'.format(m1, m2))            
        elif m1 > 0 and m2 > 0:
            left()
            print('Left     : [  {:.4f},  {:.4f} ]'.format(m1, m2)) 
        elif m1 < 0 and m2 > 0:
            straight()
            print('Straight : [  {:.4f}, {:.4f} ]'.format(m1, m2)) 
        else:
            slow()
            print('Slow     : [  {:.4f},  {:.4f} ]'.format(m1, m2)) 
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

In [304]:
main()

5
4
3
2
1
Left     : [  1.0000,  0.2664 ]
Left     : [  0.5795,  1.0000 ]
Left     : [  1.0000,  0.5753 ]
Left     : [  0.5759,  1.0000 ]
Left     : [  0.5322,  1.0000 ]
Left     : [  0.5333,  1.0000 ]
Left     : [  0.5322,  1.0000 ]
Left     : [  0.6000,  0.0526 ]
Left     : [  0.5786,  1.0000 ]
Left     : [  0.5322,  1.0000 ]
Left     : [  0.5304,  1.0000 ]
Left     : [  1.0000,  0.5294 ]
Left     : [  1.0000,  0.0843 ]
Slow     : [  1.0000,  -1.0000 ]
Left     : [  1.0000,  0.6257 ]
Left     : [  1.0000,  0.6125 ]
Slow     : [  1.0000,  -1.0000 ]
Left     : [  1.0000,  0.5775 ]
Left     : [  1.0000,  0.5556 ]
Left     : [  1.0000,  0.5764 ]
Left     : [  1.0000,  0.6125 ]
Straight : [  -0.6009, 1.0000 ]
Straight : [  -0.5084, 0.0324 ]
Right    : [ -0.5000, -0.6023 ]
Slow     : [  0.0000,  0.0000 ]
Straight : [  -0.6027, 0.0893 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.1037,  -0.6732 ]
Straight : [  -0.6732, 0.1223 ]
Straight : [  -0.6456, 0.0916 ]
Right    : [ -0.7000, -0.00

Slow     : [  0.5770,  -1.0000 ]
Straight : [  -1.0000, 0.6014 ]
Slow     : [  0.4663,  -1.0000 ]
Slow     : [  0.4286,  -1.0000 ]
Left     : [  0.4887,  0.6788 ]
Left     : [  0.5339,  0.2122 ]
Left     : [  0.5766,  0.2460 ]
Straight : [  -0.7000, 1.0000 ]
Straight : [  -0.5789, 1.0000 ]
Straight : [  -0.5577, 1.0000 ]
Straight : [  -0.5794, 1.0000 ]
Straight : [  -0.6277, 1.0000 ]
Straight : [  -0.5811, 1.0000 ]
Straight : [  -0.6498, 0.7817 ]
Straight : [  -0.6250, 0.7788 ]
Right    : [ -1.0000, -0.5549 ]
Straight : [  -1.0000, 0.8400 ]
Slow     : [  0.2294,  -1.1918 ]
Slow     : [  0.2297,  -1.3308 ]
Straight : [  -1.1915, 0.2290 ]
Left     : [  0.2326,  0.6476 ]
Slow     : [  0.2305,  -1.0000 ]
Left     : [  0.7009,  0.2685 ]
Left     : [  0.2710,  0.7269 ]
Left     : [  0.2727,  0.6753 ]
Straight : [  -1.0000, 0.2941 ]
Straight : [  -0.6478, 1.0000 ]
Straight : [  -0.7000, 1.0000 ]
Right    : [ -1.0000, -0.7277 ]
Straight : [  -1.0000, 0.7263 ]
Straight : [  -1.0000, 0.6484 ]
St

Slow     : [  0.7821,  -1.0000 ]
Straight : [  -1.0000, 0.7000 ]
Straight : [  -1.0000, 0.7255 ]
Straight : [  -1.0000, 0.7798 ]
Slow     : [  0.7794,  -1.0000 ]
Left     : [  0.4658,  0.7782 ]
Left     : [  1.0000,  0.4242 ]
Straight : [  -0.6489, 1.3272 ]
Straight : [  -0.4661, 0.0435 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Straight : [  -0.5333, 0.2470 ]
Slow     : [  0.0000,  0.0000 ]
Straight : [  -0.5987, 0.2455 ]
Straight : [  -0.6222, 0.2312 ]
Straight : [  -0.6778, 0.2857 ]
Left     : [  0.3091,  0.2457 ]
Left     : [  0.2472,  0.3064 ]
Left     : [  0.2300,  0.1739 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Left     : [  0.2308,  0.2880 ]
Left     : [  0.2488,  0.3017 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Straight : [  -0.7765, 0.2338 ]
Straight : [  -0.7500, 0.2484 ]
Slow     : [  0.0000,  0.0000 ]
Slow  

Left     : [  0.1754,  0.2675 ]
Left     : [  0.2500,  0.1761 ]
Slow     : [  0.2687,  -1.4839 ]
Slow     : [  0.2500,  -3.5294 ]
Left     : [  0.3235,  0.2324 ]
Straight : [  -4.0667, 0.2483 ]
Slow     : [  0.3056,  -6.1724 ]
Slow     : [  0.2876,  -2.4861 ]
Slow     : [  0.3067,  -2.4722 ]
Slow     : [  0.2862,  -2.3784 ]
Slow     : [  0.3279,  -11.3750 ]
Slow     : [  0.3235,  -4.0417 ]
Slow     : [  0.3077,  -2.0595 ]
Left     : [  0.3692,  0.1579 ]
Slow     : [  0.2697,  -3.4815 ]
Slow     : [  0.3259,  -1.8889 ]
Straight : [  -1.5981, 0.3807 ]
Slow     : [  0.3265,  -1.6038 ]
Straight : [  -1.7979, 0.3242 ]
Straight : [  -1.7347, 0.3037 ]
Slow     : [  0.3632,  -1.6000 ]
Slow     : [  0.3467,  -1.6667 ]
Slow     : [  0.3652,  -2.7385 ]
Right    : [ -2.4667, -4.0000 ]
Slow     : [  0.3644,  -2.4730 ]
Straight : [  -3.0678, 0.3429 ]
Straight : [  -2.9000, 0.2690 ]
Straight : [  -2.3553, 0.4030 ]
Slow     : [  0.3626,  -5.5937 ]
Straight : [  -2.0455, 0.2696 ]
Slow     : [  0.3437, 

Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow    

Straight : [  -0.4466, 0.0758 ]
Straight : [  -0.5545, 0.2727 ]
Straight : [  -1.0000, 0.0769 ]
Straight : [  -1.0000, 0.0769 ]
Straight : [  -1.0000, 0.3840 ]
Straight : [  -1.0000, 0.3840 ]
Straight : [  -1.0000, 0.3840 ]
Straight : [  -1.0000, 0.4661 ]
Straight : [  -1.0000, 0.4661 ]
Straight : [  -1.0000, 0.3840 ]
Straight : [  -1.0000, 0.3840 ]
Straight : [  -1.0000, 0.3840 ]
Straight : [  -1.0000, 0.3047 ]
Straight : [  -1.0000, 0.3639 ]
Straight : [  -1.0000, 0.3639 ]
Straight : [  -1.0000, 0.3466 ]
Straight : [  -1.0000, 0.3466 ]
Slow     : [  0.3820,  -1.0000 ]
Slow     : [  0.3820,  -1.0000 ]
Left     : [  0.4044,  0.0714 ]
Straight : [  -1.0000, 0.3658 ]
Straight : [  -1.0000, 0.3658 ]
Left     : [  0.4659,  0.3864 ]
Left     : [  0.4659,  0.3864 ]
Left     : [  0.4257,  0.0751 ]
Slow     : [  0.4247,  -1.0000 ]
Straight : [  -1.0000, 0.4659 ]
Straight : [  -1.0000, 0.4659 ]
Slow     : [  0.4464,  -1.0000 ]
Slow     : [  0.4464,  -1.0000 ]
Straight : [  -1.0000, 0.5292 ]
Slo

Straight : [  -1.0000, 0.6282 ]
Straight : [  -1.0000, 0.6250 ]
Straight : [  -1.0000, 0.0862 ]
Straight : [  -1.0000, 0.0781 ]
Straight : [  -1.0000, 0.0710 ]
Straight : [  -1.0000, 0.0710 ]
Right    : [ -1.0000, -0.5320 ]
Right    : [ -0.4885, -1.0000 ]
Right    : [ -0.5286, -1.0000 ]
Right    : [ -0.4692, -1.0000 ]
Right    : [ -0.5115, -1.0000 ]
Right    : [ -0.4905, -1.0000 ]
Right    : [ -0.4905, -1.0000 ]
Right    : [ -0.5116, -1.0000 ]
Straight : [  -0.5321, 0.8099 ]
Right    : [ -1.0000, -0.5120 ]
Right    : [ -1.0000, -0.5120 ]
Straight : [  -1.0000, 1.0000 ]
Straight : [  -0.4877, 0.5820 ]
Right    : [ -0.5298, -1.0000 ]
Right    : [ -0.5298, -1.0000 ]
Right    : [ -0.7240, -1.0000 ]
Straight : [  -1.0000, 1.0000 ]
Straight : [  -1.0000, 1.0000 ]
Straight : [  -0.7526, 0.6049 ]
Right    : [ -1.0000, -0.7481 ]
Right    : [ -0.6748, -1.0000 ]
Right    : [ -0.7513, -1.0000 ]
Right    : [ -0.6970, -1.0000 ]
Right    : [ -0.6970, -1.0000 ]
Right    : [ -0.6732, -1.0000 ]
Right   

Straight : [  -0.6513, 0.0833 ]
Straight : [  -0.7246, 0.0893 ]
Straight : [  -0.7246, 0.0893 ]
Straight : [  -0.7246, 0.0893 ]
Straight : [  -0.8711, 0.2479 ]
Straight : [  -0.8711, 0.2479 ]
Left     : [  0.3436,  0.2292 ]
Left     : [  0.3436,  0.2292 ]
Left     : [  0.3436,  0.2292 ]
Left     : [  0.3436,  0.2292 ]
Slow     : [  0.2305,  -1.0000 ]
Slow     : [  0.2305,  -1.0000 ]
Slow     : [  0.2458,  -1.0000 ]
Slow     : [  0.2458,  -1.0000 ]
Slow     : [  0.2458,  -1.0000 ]
Slow     : [  0.2305,  -1.0000 ]
Slow     : [  0.2305,  -1.0000 ]
Slow     : [  0.3835,  -0.8723 ]
Slow     : [  0.3835,  -0.8723 ]
Slow     : [  0.2657,  -0.7837 ]
Slow     : [  0.2657,  -0.7837 ]
Slow     : [  0.2657,  -0.7837 ]
Straight : [  -0.2921, 0.2843 ]
Straight : [  -0.2921, 0.2843 ]
Right    : [ -0.6754, -0.2800 ]
Right    : [ -0.6754, -0.2800 ]
Right    : [ -0.6754, -0.2800 ]
Right    : [ -0.2917, -0.6770 ]
Straight : [  -0.2982, 0.2869 ]
Straight : [  -0.2982, 0.2869 ]
Right    : [ -0.3000, -0.702

Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow     : [  0.0000,  0.0000 ]
Slow    